In [1]:
import joblib
from statsmodels.stats.weightstats import ttest_ind
from statsmodels.stats.weightstats import DescrStatsW
import numpy as np
import pandas as pd
from ALSampler import summarize
from itertools import chain

In [2]:
m_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_crf_scores_sentenceSampling.joblib')

In [3]:
r_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_random_scores_sentenceSampling.joblib')

In [4]:
vmax_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_VBmax_scores_sentenceSampling_fixed.joblib')

In [5]:
vmin_scores=joblib.load(r'../data/n2c2/scores_sentence_sampling/ner_VBmin_scores_sentenceSampling_fixed.joblib')

In [6]:
name1='m'
name2='v'
measures=['precision','recall','f1'] 
cols=[b+"_"+a for b in measures for a in [name1, name2,'pv'] ]
results={col:[] for col in cols}

In [7]:
results

{'precision_m': [],
 'precision_v': [],
 'precision_pv': [],
 'recall_m': [],
 'recall_v': [],
 'recall_pv': [],
 'f1_m': [],
 'f1_v': [],
 'f1_pv': []}

In [8]:
def compute_stats(scores1, scores2, name1, name2, runs=3, rounds=9):
    measures=['precision','recall','f1'] 
    cols=[b+"_"+a for b in measures for a in [name1, name2,'pv'] ]
    results={col:[] for col in cols}
    for ri in range(rounds):
        single_round_scores={name1:{}, name2:{}}
        for m in measures:             
            # print('s1',len(scores1), len(scores1[0]), len(scores1[0][0][m]))
            # print('s2',len(scores2), len(scores2[0]), len(scores2[0][0][m]))
            single_round_scores[name1][m]=list(chain.from_iterable([scores1[u][ri][m] for u in range(runs)]))
            single_round_scores[name2][m]=list(chain.from_iterable([scores2[u][ri][m] for u in range(runs)]))
            results[m+'_'+name1].append(np.mean(single_round_scores[name1][m]))
            results[m+'_'+name2].append(np.mean(single_round_scores[name2][m]))
            ttres=ttest_ind(single_round_scores[name1][m], single_round_scores[name2][m], usevar='unequal')
            # ttres=DescrStatsW.ttest_mean(np.array(single_round_scores[name1][m])-np.array(single_round_scores[name2][m]))
            results[m+'_pv'].append(ttres[1])
    return pd.DataFrame(results)

In [9]:
list(range(3))

[0, 1, 2]

### model vs vmin

In [10]:
results=compute_stats(m_scores, vmin_scores, 'm', 'vmin')

In [11]:
results

,precision_m,precision_vmin,precision_pv,recall_m,recall_vmin,recall_pv,f1_m,f1_vmin,f1_pv
0,0.941698,0.941698,1.000000e+00,0.784626,0.784626,1.000000e+00,0.856001,0.856001,1.000000e+00
1,0.944734,0.937885,0.000000e+00,0.812382,0.824754,0.000000e+00,0.873567,0.877683,1.329039e-161
2,0.945090,0.940066,7.818046e-283,0.829300,0.839625,0.000000e+00,0.883415,0.887009,1.093828e-140
3,0.943034,0.941677,2.682182e-36,0.840918,0.846989,3.333232e-171,0.889050,0.891823,2.434472e-99
4,0.944085,0.943112,2.614449e-22,0.847206,0.849625,3.574958e-32,0.893021,0.893928,3.554695e-13
5,0.943927,0.943932,9.583599e-01,0.850443,0.854094,1.339006e-84,0.894746,0.896766,1.058488e-62
6,0.944762,0.944202,2.084413e-07,0.853562,0.859061,5.125559e-176,0.896847,0.899619,3.676803e-109
7,0.945369,0.944859,6.144929e-06,0.854485,0.858924,7.684329e-121,0.897629,0.899842,1.793283e-77
8,0.946330,0.944372,5.030275e-63,0.856412,0.859315,3.014067e-60,0.899126,0.899835,1.685625e-10


### model vs vmax

In [13]:
results=compute_stats(m_scores, vmax_scores, 'm', 'vmax')
results

,precision_m,precision_vmax,precision_pv,recall_m,recall_vmax,recall_pv,f1_m,f1_vmax,f1_pv
0,0.941698,0.946086,1.750851e-178,0.784626,0.771826,1.491339e-259,0.856001,0.850114,1.783639e-196
1,0.944734,0.937867,0.000000e+00,0.812382,0.825620,0.000000e+00,0.873567,0.878168,2.869682e-191
2,0.945090,0.942118,3.855048e-146,0.829300,0.836394,5.373877e-240,0.883415,0.886111,1.572515e-100
3,0.943034,0.941730,1.456067e-36,0.840918,0.844936,1.011478e-97,0.889050,0.890709,8.881833e-42
4,0.944085,0.944626,4.636057e-08,0.847206,0.848992,3.984855e-21,0.893021,0.894257,1.836403e-24
5,0.943927,0.944928,2.904583e-22,0.850443,0.854722,1.148649e-112,0.894746,0.897562,5.602072e-112
6,0.944762,0.945918,2.432976e-29,0.853562,0.857886,4.427537e-128,0.896847,0.899752,6.425807e-123
7,0.945369,0.945305,5.535977e-01,0.854485,0.859243,1.620336e-138,0.897629,0.900219,1.998186e-100
8,0.946330,0.944848,5.242270e-52,0.856412,0.859267,2.421263e-66,0.899126,0.900026,4.666625e-17


### model vs random

In [13]:
results=compute_stats(m_scores, r_scores, 'm', 'r')
results

,precision_m,precision_r,precision_pv,recall_m,recall_r,recall_pv,f1_m,f1_r,f1_pv
0,0.941698,0.941698,1.000000e+00,0.784626,0.784626,1.000000e+00,0.856001,0.856001,1.000000e+00
1,0.944734,0.942639,9.803614e-72,0.812382,0.819758,1.290575e-141,0.873567,0.876905,3.733907e-97
2,0.945090,0.941645,1.358397e-136,0.829300,0.833949,1.833995e-87,0.883415,0.884524,2.849791e-15
3,0.943034,0.944257,4.149644e-21,0.840918,0.840774,5.499830e-01,0.889050,0.889504,8.987194e-04
4,0.944085,0.944411,2.005027e-03,0.847206,0.846915,1.401401e-01,0.893021,0.893006,9.020093e-01
5,0.943927,0.944874,7.627020e-14,0.850443,0.850365,6.696785e-01,0.894746,0.895126,7.483581e-04
6,0.944762,0.944773,9.183713e-01,0.853562,0.853923,2.468465e-02,0.896847,0.897051,6.365473e-02
7,0.945369,0.945981,4.623089e-09,0.854485,0.855405,1.345725e-07,0.897629,0.898413,3.339954e-12
8,0.946330,0.945812,3.319102e-08,0.856412,0.857211,1.243113e-06,0.899126,0.899332,5.953043e-02


### vmin vs random

In [14]:
results=compute_stats(r_scores, vmin_scores, 'r', 'v')
results

,precision_r,precision_v,precision_pv,recall_r,recall_v,recall_pv,f1_r,f1_v,f1_pv
0,0.941698,0.941698,1.000000e+00,0.784626,0.784626,1.000000e+00,0.856001,0.856001,1.000000e+00
1,0.942639,0.937885,8.179735e-211,0.819758,0.824754,1.576462e-80,0.876905,0.877683,3.218302e-08
2,0.941645,0.940066,5.320221e-35,0.833949,0.839625,4.369100e-121,0.884524,0.887009,1.028663e-62
3,0.944257,0.941677,3.435076e-77,0.840774,0.846989,1.827201e-115,0.889504,0.891823,1.871826e-60
4,0.944411,0.943112,2.245762e-32,0.846915,0.849625,2.867655e-48,0.893006,0.893928,1.136310e-15
5,0.944874,0.943932,1.630081e-14,0.850365,0.854094,1.152757e-87,0.895126,0.896766,2.269744e-47
6,0.944773,0.944202,1.856241e-08,0.853923,0.859061,9.749551e-163,0.897051,0.899619,1.013348e-102
7,0.945981,0.944859,7.064010e-28,0.855405,0.858924,1.878226e-82,0.898413,0.899842,2.565903e-36
8,0.945812,0.944372,1.030740e-36,0.857211,0.859315,1.288282e-35,0.899332,0.899835,3.319459e-06


### vmax vs random

In [15]:
results=compute_stats(r_scores, vmax_scores, 'r', 'v')
results

,precision_r,precision_v,precision_pv,recall_r,recall_v,recall_pv,f1_r,f1_v,f1_pv
0,0.941698,0.946086,1.750851e-178,0.784626,0.771826,1.491339e-259,0.856001,0.850114,1.783639e-196
1,0.942639,0.937867,1.063272e-257,0.819758,0.825620,3.121843e-109,0.876905,0.878168,4.352412e-19
2,0.941645,0.942118,8.634047e-05,0.833949,0.836394,8.562154e-31,0.884524,0.886111,1.815129e-31
3,0.944257,0.941730,1.110791e-77,0.840774,0.844936,8.336183e-63,0.889504,0.890709,3.127903e-19
4,0.944411,0.944626,4.423310e-02,0.846915,0.848992,3.521729e-35,0.893006,0.894257,1.948355e-29
5,0.944874,0.944928,6.480154e-01,0.850365,0.854722,5.460119e-116,0.895126,0.897562,4.396442e-96
6,0.944773,0.945918,6.052104e-33,0.853923,0.857886,2.286427e-114,0.897051,0.899752,3.700007e-117
7,0.945981,0.945305,2.770880e-12,0.855405,0.859243,3.437653e-98,0.898413,0.900219,7.682288e-55
8,0.945812,0.944848,4.356996e-24,0.857211,0.859267,2.948215e-39,0.899332,0.900026,2.599924e-11


#### vmax vs vmin

In [16]:
results=compute_stats(vmin_scores, vmax_scores, 'vmin', 'vmax')
results

,precision_vmin,precision_vmax,precision_pv,recall_vmin,recall_vmax,recall_pv,f1_vmin,f1_vmax,f1_pv
0,0.941698,0.946086,1.750851e-178,0.784626,0.771826,1.491339e-259,0.856001,0.850114,1.783639e-196
1,0.937885,0.937867,8.809647e-01,0.824754,0.825620,7.655080e-06,0.877683,0.878168,7.791976e-05
2,0.940066,0.942118,4.044804e-74,0.839625,0.836394,9.079326e-74,0.887009,0.886111,2.278426e-14
3,0.941677,0.941730,5.984592e-01,0.846989,0.844936,8.456556e-32,0.891823,0.890709,8.000780e-22
4,0.943112,0.944626,5.906196e-48,0.849625,0.848992,1.207748e-04,0.893928,0.894257,2.859242e-03
5,0.943932,0.944928,3.231032e-24,0.854094,0.854722,8.329432e-05,0.896766,0.897562,3.388272e-13
6,0.944202,0.945918,5.011711e-61,0.859061,0.857886,2.109234e-13,0.899619,0.899752,2.137622e-01
7,0.944859,0.945305,2.405734e-05,0.858924,0.859243,4.783617e-02,0.899842,0.900219,5.346985e-04
8,0.944372,0.944848,1.542762e-05,0.859315,0.859267,7.548925e-01,0.899835,0.900026,6.563243e-02
